In [1]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession

spark_version='3.5.3'
scala_version='2.12'
spark_connector_version='3.5.3'
kafka_client_version='3.5.1'
common_pool2_version='2.12.0'

packages = [
    f"spark-sql-kafka-0-10_{scala_version}-{spark_connector_version}.jar",
    f"kafka-clients-{kafka_client_version}.jar",
    f"commons-pool2-{common_pool2_version}.jar",
    f"spark-token-provider-kafka-0-10_{scala_version}-{spark_version}.jar"
]

SPARK_HOME = "/usr/local/spark"
SPARK_JARS_DIR = SPARK_HOME + "/jars/"

packages = [SPARK_JARS_DIR + p for p in packages]

spark = SparkSession.builder \
    .appName("kafka-example") \
    .config("spark.jars", ",".join(packages)) \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

spark

25/11/25 02:25:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 02:25:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
kafka_server = "kafka:9092" 
topic_name = "LeNguyenHoangPhuc_RandomNumber"         

streamRawDf = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", topic_name) \
    .load()


In [6]:
from pyspark.sql.functions import col
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 )

In [7]:
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()
query2 = stream_writer2.start()

25/11/25 02:25:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ba96a820-7830-4f73-8149-643fc228259c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/25 02:25:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/11/25 02:25:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f81e3f93-d189-4e97-8cb2-22878ac124f1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/25 02:25:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

In [10]:
from time import sleep
from IPython.display import display, clear_output

x = 0
try:
    while(True):
        print("Showing live view refreshed every 5 seconds")
        print("This code is set up by Le Nguyen Hoang Phuc - 23521198")
        print(f"Seconds passed: {x*2}")
        result1 = spark.sql(f"select * from {query1.name}")
        result2 = spark.sql(f"select * from {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(2)
        x += 1
        clear_output(wait=True)
except KeyboardInterrupt:
    print("break")
print("Live view ended")

Showing live view refreshed every 5 seconds
This code is set up by Le Nguyen Hoang Phuc - 23521198
Seconds passed: 20


,topic,offset,rand_number
0,LeNguyenHoangPhuc_RandomNumber,752,4
1,LeNguyenHoangPhuc_RandomNumber,753,4
2,LeNguyenHoangPhuc_RandomNumber,754,4
3,LeNguyenHoangPhuc_RandomNumber,755,4
4,LeNguyenHoangPhuc_RandomNumber,756,4
...,...,...,...
202,LeNguyenHoangPhuc_RandomNumber,954,2
203,LeNguyenHoangPhuc_RandomNumber,955,2
204,LeNguyenHoangPhuc_RandomNumber,956,2
205,LeNguyenHoangPhuc_RandomNumber,957,2


,topic,offset,rand_number,Is_Even
0,LeNguyenHoangPhuc_RandomNumber,752,4,True
1,LeNguyenHoangPhuc_RandomNumber,753,4,True
2,LeNguyenHoangPhuc_RandomNumber,754,4,True
3,LeNguyenHoangPhuc_RandomNumber,755,4,True
4,LeNguyenHoangPhuc_RandomNumber,756,4,True
...,...,...,...,...
202,LeNguyenHoangPhuc_RandomNumber,954,2,True
203,LeNguyenHoangPhuc_RandomNumber,955,2,True
204,LeNguyenHoangPhuc_RandomNumber,956,2,True
205,LeNguyenHoangPhuc_RandomNumber,957,2,True


break
Live view ended
